In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from MetabolomicsPaper.GeneralHelperFunctions import *
# from ACSPaper.Matching import *
from ACSPaper.ACSPaperUtils import *
from sklearn.metrics import roc_curve, auc
import shap
from sklearn.model_selection import train_test_split

paper_dir = '/net/mraid08/export/jafar/Microbiome/Analyses/Noamba/Cardio/ACS_study/v2/'
curr_dir = os.path.join(paper_dir, 'outcome_prediction')
make_dir_if_not_exists(curr_dir)
os.chdir(paper_dir)

In [6]:
from LabData.DataLoaders.MetabolonLoader import MetabolonLoader

In [2]:
acs_data_metabolon = pd.read_csv(os.path.join(paper_dir, 'serum_metabolomics_signatures', 'acs_data_metabolon.csv'), index_col=0)
acs_data_metabolon['combined_outcome'] = acs_data_metabolon[['Acute MI', 'Acute stroke', 'Acute limb event', 'Unplanned PCI', 'New-onset HF', 'New-onset Afib', 'CV hospitalization including chest pain', 'CV death',]].any(1)
acs_data_metabolon['Acute_MI_or_Stroke'] = acs_data_metabolon[['Acute MI', 'Acute stroke']].any(1)
acs_data_metabolon['combined_cvd_outcome'] = acs_data_metabolon[['Acute MI', 'Acute stroke', 'Unplanned PCI', 'CV death',]].any(1)

Columns (10,23,30,33,40,96,97,101,107,110,111,112,131,132,133,134,135,136,137,138,139,140,141,142,143,145,148,149,150,151,152,153,155,157,159,169,170,179,184) have mixed types.Specify dtype option on import or set low_memory=False.


In [4]:
storage_bad_metabs = pd.read_pickle(os.path.join('../data', 'storage_time_bad_metabs_bonf.pkl'))

In [7]:
metabolon = MetabolonLoader().get_data(study_ids=['PNP1', 'Cardio'], metabolon_runs=['Metabolon_MAR2017', 'Metabolon_MAY2018'],
                                       robust_zs=True, clip_outliers_std=5, clip_outliers_or_na='clip', fill_missing_with_min=True, 
                                       log_transform_base=10, norm_within_run_using_anchors=None, norm_across_run_using_anchors=True, 
                                       normalize_std=True)

In [12]:
metabolon_cols = list(set(metabolon.df.columns) - set(storage_bad_metabs))
acs_data_metabolon.loc[:, metabolon_cols].to_csv(os.path.join(curr_dir, 'acs_936metabs_X.csv'))
acs_data_metabolon.loc[:, ['combined_outcome', 'Acute_MI_or_Stroke', 'combined_cvd_outcome']].to_csv(os.path.join(curr_dir, 'acs_outcomes.csv'))

python ~/workspace/metabolon/Analyses/RSCV.py ./outcome_RSCV -n_cols_per_job 1 -n_random 5 -path_to_X /net/mraid08/export/jafar/Microbiome/Analyses/Noamba/Cardio/ACS_study/v2/outcome_prediction/acs_936metabs_X.csv -path_to_Y /net/mraid08/export/jafar/Microbiome/Analyses/Noamba/Cardio/ACS_study/v2/outcome_prediction/acs_outcomes.csv  &